In [ ]:
!pip install openprompt
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00


In [ ]:
"""
This file contains the logic for loading data for all Conditional Generation tasks.
"""

from openprompt.data_utils.utils import InputExample
import os
from typing import List, Dict

from openprompt.data_utils.data_processor import DataProcessor
import torch

import re
import pandas as pd

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [ ]:
class CLAQUAPromptProcessor(DataProcessor):

  def __init__(self, data_labels):
    super().__init__()
    #self.labels = data_labels
    self.labels = torch.tensor(data_labels, dtype=torch.float).unsqueeze(dim=1)  # convert dataframe column to tensor (convert to numpy: labels.to_numpy())

    print(len(self.labels))

  def get_examples(self, data_path: str, split: str) -> List[InputExample]:
    data_label2id = {"no":0, "yes":1}
    data_id2label = dict()
    for k,v in data_label2id.items():
      data_id2label[v] = k

    text_items = data_path.to_numpy()
    #encoded_text_items = data['enc_full_text_padded'].to_numpy()
    examples = []

    for i in range(len(text_items)):
      label = int(self.labels[i])
      full_text = text_items[i]
      context = full_text.split('[CONTEXT_SEP]')[0].strip()
      ent1 = full_text.split('[CONTEXT_SEP]')[1].split('[ENTITY_SEP]')[0].strip()
      ent2 = full_text.split('[CONTEXT_SEP]')[1].split('[ENTITY_SEP]')[1].strip()
      #src = items.iloc[i]['context'] + " [SEP] " + items.iloc[i]['entity1'] + " [SEP] " + items.iloc[i]['entity2']
      tgt = data_id2label[label]
      merged_text = "CONTEXT: " + context + " ENTITY 1: " + ent1 + " ENTITY 2: " + ent2
      # example = InputExample(guid=str(i), meta=context, text_a=ent1, text_b=ent2, tgt_text=tgt, label=label)
      example = InputExample(guid=str(i), text_a=merged_text, tgt_text=tgt, label=label)
      examples.append(example)

    return examples